In [14]:
include("../code/headers.jl")

In [2]:
n = factorial(7)*3
k = 3
R = 1 .- 2 ./k
m_ = n*(1 .- R)
m = Int.(round.(m_))
m_

10080.0

In [5]:
randseed = 123
algo = MS(maxiter=1000, Tmax=1, gamma=0.0, sigma=1e-10)
Y = [rand([0,1], n) for mm in m]

0-dimensional Array{Array{Int64,1},0}:
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1  …  0, 1, 1, 0, 0, 0, 0, 1, 1, 1]

In [15]:
res = Vector{BPResults{MS}}(undef, length(m))
msgs = [Float64[] for _ in m]
fgs = Vector{FactorGraph}(undef,length(m))
for j in eachindex(m)
    println("Rate $(round(R[j],digits=2)). $j of $(length(m))")
    fg = ldpc_graph(Val(2), n, m[j], randseed=randseed)
    extfields!(fg,Y[j],algo,randseed=randseed+j)
    res[j] = bp!(fg, algo, Y[j], verbose=true)
    msgs[j] = [ms[end] for ms in fg.mfv]
    fgs[j] = deepcopy(fg)
end

Rate 0.33. 1 of 1


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:15


In [36]:
fg = fgs[1]
y = Y[1]
msg = msgs[1]
# Round messages
msg_rounded = round.(Int, msg)
@show maximum(abs,msg-msg_rounded)
s = 2*y .- 1
σ = guesses(fg)
σ_rounded = round.(Int, x)
ovl_rounded = dot(σ_rounded, s) / n
dist_rounded = 0.5*(1 - ovl_rounded)
distortion(fg,y)

maximum(abs, msg - msg_rounded) = 2.205654325848627e-9


0.21104497354497354

In [16]:
plts = []
bins = -2.25:0.5:2.25
levels = -6:6
cnts = []
dist = zeros(length(m))
for j in eachindex(m)
    push!(plts, histogram(msgs[j], label="k=$(k[j])", bins=bins, normalize=:pdf))
    push!(cnts, OffsetArray(StatsBase.proportions(Int.(msgs[j]), levels)))
    dist[j] = distortion(fgs[j], Y[j])
end
plot(plts..., layout=(2,4), size=(800,400))

LoadError: InexactError: Int64(8.081377789181943e-10)

In [17]:
cnts[1]

LoadError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

In [24]:
[k dist]

8×2 Array{Float64,2}:
  3.0  0.249206
  4.0  0.187302
  5.0  0.185317
  6.0  0.1875
  7.0  0.173413
  8.0  0.128968
  9.0  0.127976
 10.0  0.109524